In [37]:
###############
### imports ###
###############


import numpy as np
import pandas as pd
import requests
import os



import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, inspect


###########################
### pulling fresh texts ###
###########################


engine = db.create_engine('sqlite:////Users/michaelharnett/Library/Messages/chat.db')
connection = engine.connect()
metadata = db.MetaData()

orig_df = pd.read_sql("select distinct m.rowid ,m.is_from_me IsFromMe ,case when m.is_from_me = 1 then m.account else h.id end as FromPhoneNumber ,datetime((m.date / 1000000000) + 978307200, 'unixepoch', 'localtime') as TextDate, m.text MessageText,c.display_name RoomName from message as m left join handle as h on m.handle_id = h.rowid left join chat as c on m.cache_roomnames = c.room_name left join chat_handle_join as ch on c.rowid = ch.chat_id left join handle as h2 on ch.handle_id = h2.rowid where RoomName LIKE 'Wordle Fam%' order by m.date desc;", engine)






#############################
### making correct tables ###
#############################


df = orig_df[['FromPhoneNumber','TextDate','MessageText']] # taking only the fields I need
df = df.rename(columns = {'FromPhoneNumber':'player', 'TextDate':'date', 'MessageText':'message'}) #fixing names


# Renaming changing the Id from phone number to our names
df.loc[df.player == '+12014523662', 'player'] = 'Danielle'
df.loc[df.player == '+12017253072', 'player'] = 'Dad'
df.loc[df.player == '+15514868670', 'player'] = 'Chris'
df.loc[df.player == 'E:michaelcharnett@gmail.com', 'player'] = 'Michael'


# Creating columns
df['wordle_check'] = ''
df['game_num'] = ''
df['score'] = ''

# fixing indexes
df = df.dropna()
df = df.reset_index()
df =df.drop(columns = ['index'])

#updating values
for i in range(1,len(df)):
    df.loc[i,'wordle_check'] = df.loc[i,'message'][:6]
    df.loc[i,'game_num'] = df.loc[i,'message'][7:10]
    df.loc[i,'score'] = df.loc[i,'message'][11:14]
    

wordledf = df    
wordledf = df[df.wordle_check == 'Wordle']






#######################
### cleaneingtables ###
#######################


## for some reason was erroring..not needed these columns were dropped elsewhere
# # removing unneeded columns
# wordledf = wordledf.drop(columns = ['message', 'date'])
# wordledf = wordledf.drop(columns = ['wordle_check'])


# setting wins/loss in new column    -----------------MAY BE UNNECCESRY 
wordledf[wordledf.score == 'x']
wordledf[wordledf.score == 'X']    #there seems to be case inconsistencies 

wordledf.loc[wordledf.score == 'x','LOSS'] = 1
wordledf.loc[wordledf.score != 'x','LOSS'] = 0

wordledf.loc[wordledf.score == 'X','LOSS'] = 1
wordledf.loc[wordledf.score != 'X','LOSS'] = 0



# individualized tables
michaeldf = wordledf[wordledf.player == 'Michael']
danielledf = wordledf[wordledf.player == 'Danielle']
chrisdf = wordledf[wordledf.player == 'Chris']
daddf = wordledf[wordledf.player == 'Dad']


# ordering the tables
michaeldf = michaeldf.sort_values(by='game_num', ascending = False)
danielledf = danielledf.sort_values(by='game_num', ascending = False)
chrisdf = chrisdf.sort_values(by='game_num', ascending = False)
daddf = daddf.sort_values(by='game_num', ascending = False)

# making a df of just wins to compare against
michaelwindf = michaeldf[michaeldf['score'].astype(str).str.isnumeric()]
dadwindf = daddf[daddf['score'].astype(str).str.isnumeric()]
daniellewindf = danielledf[danielledf['score'].astype(str).str.isnumeric()]
chriswindf = chrisdf[chrisdf['score'].astype(str).str.isnumeric()]



#######################################################################################
########    REALIZING, tables should be made and exprted. thsi can be dane after ######
########################################################################################
#### Win percentages 
#### michael_winpercent = (len(michaelwindf)/len(michaeldf)*100)
#### dad_winpercent = (len(dadwindf)/len(daddf)*100)
#### danielle_winpercent = (len(daniellewindf)/len(danielledf)*100)
#### chris_winpercent = (len(chriswindf)/len(chrisdf)*100)


##############################
#### saving dfs to folder ####
##############################

# michael
michaeldf.to_csv('tables/michaeldf.csv', index = False)
michaelwindf.to_csv('tables/michaelwindf.csv', index = False)

# danielle
danielledf.to_csv('tables/danielledf.csv', index = False)
daniellewindf.to_csv('tables/daniellewindf.csv', index = False)

#chris
chrisdf.to_csv('tables/chrisdf.csv', index = False)
chriswindf.to_csv('tables/chriswindf.csv', index = False)

#dad
daddf.to_csv('tables/daddf.csv', index = False)
dadwindf.to_csv('tables/dadwindf.csv', index = False)

/Users/michaelharnett/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/Users/michaelharnett/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [38]:
michaeldf

,player,date,message,wordle_check,game_num,score,LOSS
2,Michael,2022-04-01 22:28:27,Wordle 286 6/6\n\n⬛⬛🟨⬛⬛\n🟨🟨🟨⬛🟨\n🟩⬛🟩🟩🟩\n🟩⬛🟩🟩🟩\n...,Wordle,286,6/6,0.0
7,Michael,2022-03-31 08:03:23,Wordle 285 4/6\n\n⬛🟨⬛⬛⬛\n🟩⬛⬛⬛⬛\n🟩🟩⬛⬛🟩\n🟩🟩🟩🟩🟩,Wordle,285,4/6,0.0
29,Michael,2022-03-30 08:49:55,Wordle 284 2/6\n\n🟩⬛⬛⬛🟨\n🟩🟩🟩🟩🟩\n\nMuahaha,Wordle,284,2/6,0.0
32,Michael,2022-03-29 08:40:05,Wordle 283 5/6\n\n🟩⬛🟩⬛⬛\n🟩⬛🟩⬛⬛\n🟩⬛🟩⬛⬛\n🟩⬛🟩⬛⬛\n...,Wordle,283,5/6,0.0
36,Michael,2022-03-28 14:32:00,Wordle 282 4/6\n\n⬛⬛⬛⬛⬛\n⬛⬛🟨⬛⬛\n⬛🟩🟩🟩🟩\n🟩🟩🟩🟩🟩,Wordle,282,4/6,0.0
57,Michael,2022-03-26 11:51:02,Wordle 280 5/6\n\n🟨⬛⬛⬛⬛\n⬛🟩⬛⬛🟨\n🟨🟩🟨⬛⬛\n⬛🟩⬛🟨⬛\n...,Wordle,280,5/6,0.0
66,Michael,2022-03-25 19:05:29,Wordle 279 4/6\n\n🟨🟩⬛⬛⬛\n⬛🟩⬛🟨⬛\n🟨🟩🟩⬛🟨\n🟩🟩🟩🟩🟩\n...,Wordle,279,4/6,0.0
71,Michael,2022-03-24 08:33:29,Wordle 278 5/6\n\n⬛⬛🟨⬛🟨\n🟨🟨⬛⬛🟨\n🟩🟩⬛⬛🟨\n🟩🟩🟩🟩⬛\n...,Wordle,278,5/6,0.0
76,Michael,2022-03-23 08:52:27,Wordle 277 X/6\n\n⬛🟨⬛🟨⬛\n🟨⬛⬛🟨⬛\n⬛⬛🟨⬛🟨\n🟨⬛⬛🟩🟩\n...,Wordle,277,X/6,0.0
84,Michael,2022-03-22 08:31:57,Wordle 276 3/6\n\n🟨⬛⬛⬛⬛\n🟩⬛🟨🟨🟨\n🟩🟩🟩🟩🟩,Wordle,276,3/6,0.0


In [69]:

# ordering the tables
michaeldf = michaeldf.sort_values(by='game_num', ascending = False)
danielledf = danielledf.sort_values(by='game_num', ascending = False)
chrisdf = chrisdf.sort_values(by='game_num', ascending = False)
daddf = daddf.sort_values(by='game_num', ascending = False)

#


In [36]:
michaeldf

,player,game_num,score,LOSS
116,Michael,’t,ven,0.0
275,Michael,’t,ott,0.0
241,Michael,zed,“I,0.0
15,Michael,yin,,0.0
458,Michael,y b,cau,0.0
...,...,...,...,...
482,Michael,,,0.0
493,Michael,,,0.0
515,Michael,,,0.0
546,Michael,,,0.0


In [47]:
michaeldf = pd.read_csv('tables/michaeldf.csv')
michaelwindf = pd.read_csv('tables/michaelwindf.csv')

In [61]:
michaeldf

,player,game_num,score,LOSS
0,Michael,286,6/6,0.0
1,Michael,285,4/6,0.0
2,Michael,284,2/6,0.0
3,Michael,283,5/6,0.0
4,Michael,282,4/6,0.0
5,Michael,280,5/6,0.0
6,Michael,279,4/6,0.0
7,Michael,278,5/6,0.0
8,Michael,277,X/6,0.0
9,Michael,276,3/6,0.0


In [67]:
michaeldf

,player,game_num,score,LOSS
0,Michael,286,6/6,0.0
1,Michael,285,4/6,0.0
2,Michael,284,2/6,0.0
3,Michael,283,5/6,0.0
4,Michael,282,4/6,0.0
5,Michael,280,5/6,0.0
6,Michael,279,4/6,0.0
7,Michael,278,5/6,0.0
8,Michael,277,X/6,0.0
9,Michael,276,3/6,0.0


In [66]:
michaeldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   player    50 non-null     object 
 1   game_num  50 non-null     int64  
 2   score     50 non-null     object 
 3   LOSS      50 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.7+ KB


In [65]:
michaeldf.score.astype(str).str.isnumeric()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
Name: score, dtype: bool

In [50]:
michaelwindf = michaeldf[michaeldf['score'].astype(str).str.isnumeric()]

dadwindf = daddf[daddf['score'].astype(str).str.isnumeric()]

daniellewindf = danielledf[danielledf['score'].astype(str).str.isnumeric()]

chriswindf = chrisdf[chrisdf['score'].astype(str).str.isnumeric()]

In [51]:
michaelwindf

,player,game_num,score,LOSS


In [55]:
michaelwindf = michaeldf[michaeldf['score'].astype(str).str.isnumeric()]

michaelwindf

In [58]:
michaelwindf = michaeldf[michaeldf['score'].astype(str).str.isnumeric()]

In [59]:
michaelwindf

,player,game_num,score,LOSS


In [60]:

michaeldf[michaeldf['score'].astype(str).str.isnumeric()]



,player,game_num,score,LOSS


In [75]:
wordledf

,player,date,message,wordle_check,game_num,score,LOSS
1,Dad,2022-04-02 01:54:03,Wordle 287 4/6\n\n⬜⬜🟨⬜🟩\n🟨🟩⬜⬜🟩\n🟩🟩⬜⬜🟩\n🟩🟩🟩🟩🟩,Wordle,287,4/6,0.0
2,Michael,2022-04-01 22:28:27,Wordle 286 6/6\n\n⬛⬛🟨⬛⬛\n🟨🟨🟨⬛🟨\n🟩⬛🟩🟩🟩\n🟩⬛🟩🟩🟩\n...,Wordle,286,6/6,0.0
3,Chris,2022-04-01 10:26:00,Wordle 286 4/6\n\n⬛️⬛️🟨⬛️⬛️\n⬛️🟨🟨🟨⬛️\n🟩🟨🟨🟨⬛️\n...,Wordle,286,4/6,0.0
4,Danielle,2022-04-01 08:28:17,Wordle 286 4/6\n\n⬛🟨🟨🟨⬛\n🟩🟨🟩🟩⬛\n🟩⬛🟩🟩🟩\n🟩🟩🟩🟩🟩,Wordle,286,4/6,0.0
5,Dad,2022-04-01 00:30:15,Wordle 286 5/6\n\n⬜⬜🟩🟩⬜\n⬜⬜🟩🟩🟩\n⬜⬜🟩🟩🟩\n🟩⬜🟩🟩🟩\n...,Wordle,286,5/6,0.0
...,...,...,...,...,...,...,...
628,Danielle,2022-02-05 11:54:27,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n⬛🟨⬛⬛🟩\n🟩⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Wordle,231,4/6,0.0
629,Michael,2022-02-05 10:31:55,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n🟩🟩⬛⬛⬛\n🟩🟩🟩⬛🟨\n🟩🟩🟩🟩🟩,Wordle,231,4/6,0.0
632,Michael,2022-02-04 08:41:31,Wordle 230 5/6\n\n🟨🟨⬛⬛⬛\n🟨🟩⬛⬛🟨\n⬛🟩🟩⬛🟨\n🟩🟩🟩🟩⬛\n...,Wordle,230,5/6,0.0
633,Danielle,2022-02-04 08:39:54,Wordle 230 4/6\n\n⬛🟨⬛⬛🟨\n⬛🟨🟨🟨⬛\n🟩🟨🟨🟨🟩\n🟩🟩🟩🟩🟩,Wordle,230,4/6,0.0


In [76]:
# converting score to single digit
for i in range(len(wordledf)):
    wordledf.loc[i, 'score'] = wordledf.loc[i, 'score'][0]

KeyError: 0

In [73]:
michaeldf = pd.read_csv('tables/michaeldf.csv')

In [91]:
wordledf.reset_index()

# for i in range(len(wordledf)):
#     wordledf.loc[i, 'score'] = wordledf.loc[i, 'score'][0]
wordledf

,player,date,message,wordle_check,game_num,score,LOSS
1,Dad,2022-04-02 01:54:03,Wordle 287 4/6\n\n⬜⬜🟨⬜🟩\n🟨🟩⬜⬜🟩\n🟩🟩⬜⬜🟩\n🟩🟩🟩🟩🟩,Wordle,287,4/6,0.0
2,Michael,2022-04-01 22:28:27,Wordle 286 6/6\n\n⬛⬛🟨⬛⬛\n🟨🟨🟨⬛🟨\n🟩⬛🟩🟩🟩\n🟩⬛🟩🟩🟩\n...,Wordle,286,6/6,0.0
3,Chris,2022-04-01 10:26:00,Wordle 286 4/6\n\n⬛️⬛️🟨⬛️⬛️\n⬛️🟨🟨🟨⬛️\n🟩🟨🟨🟨⬛️\n...,Wordle,286,4/6,0.0
4,Danielle,2022-04-01 08:28:17,Wordle 286 4/6\n\n⬛🟨🟨🟨⬛\n🟩🟨🟩🟩⬛\n🟩⬛🟩🟩🟩\n🟩🟩🟩🟩🟩,Wordle,286,4/6,0.0
5,Dad,2022-04-01 00:30:15,Wordle 286 5/6\n\n⬜⬜🟩🟩⬜\n⬜⬜🟩🟩🟩\n⬜⬜🟩🟩🟩\n🟩⬜🟩🟩🟩\n...,Wordle,286,5/6,0.0
...,...,...,...,...,...,...,...
628,Danielle,2022-02-05 11:54:27,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n⬛🟨⬛⬛🟩\n🟩⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Wordle,231,4/6,0.0
629,Michael,2022-02-05 10:31:55,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n🟩🟩⬛⬛⬛\n🟩🟩🟩⬛🟨\n🟩🟩🟩🟩🟩,Wordle,231,4/6,0.0
632,Michael,2022-02-04 08:41:31,Wordle 230 5/6\n\n🟨🟨⬛⬛⬛\n🟨🟩⬛⬛🟨\n⬛🟩🟩⬛🟨\n🟩🟩🟩🟩⬛\n...,Wordle,230,5/6,0.0
633,Danielle,2022-02-04 08:39:54,Wordle 230 4/6\n\n⬛🟨⬛⬛🟨\n⬛🟨🟨🟨⬛\n🟩🟨🟨🟨🟩\n🟩🟩🟩🟩🟩,Wordle,230,4/6,0.0


In [93]:
wordledf = wordledf.reset_index()
wordledf = wordledf.drop(columns = ['index'])

In [94]:
for i in range(len(wordledf)):
    wordledf.loc[i, 'score'] = wordledf.loc[i, 'score'][0]

In [95]:
wordledf

,player,date,message,wordle_check,game_num,score,LOSS
0,Dad,2022-04-02 01:54:03,Wordle 287 4/6\n\n⬜⬜🟨⬜🟩\n🟨🟩⬜⬜🟩\n🟩🟩⬜⬜🟩\n🟩🟩🟩🟩🟩,Wordle,287,4,0.0
1,Michael,2022-04-01 22:28:27,Wordle 286 6/6\n\n⬛⬛🟨⬛⬛\n🟨🟨🟨⬛🟨\n🟩⬛🟩🟩🟩\n🟩⬛🟩🟩🟩\n...,Wordle,286,6,0.0
2,Chris,2022-04-01 10:26:00,Wordle 286 4/6\n\n⬛️⬛️🟨⬛️⬛️\n⬛️🟨🟨🟨⬛️\n🟩🟨🟨🟨⬛️\n...,Wordle,286,4,0.0
3,Danielle,2022-04-01 08:28:17,Wordle 286 4/6\n\n⬛🟨🟨🟨⬛\n🟩🟨🟩🟩⬛\n🟩⬛🟩🟩🟩\n🟩🟩🟩🟩🟩,Wordle,286,4,0.0
4,Dad,2022-04-01 00:30:15,Wordle 286 5/6\n\n⬜⬜🟩🟩⬜\n⬜⬜🟩🟩🟩\n⬜⬜🟩🟩🟩\n🟩⬜🟩🟩🟩\n...,Wordle,286,5,0.0
...,...,...,...,...,...,...,...
202,Danielle,2022-02-05 11:54:27,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n⬛🟨⬛⬛🟩\n🟩⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Wordle,231,4,0.0
203,Michael,2022-02-05 10:31:55,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n🟩🟩⬛⬛⬛\n🟩🟩🟩⬛🟨\n🟩🟩🟩🟩🟩,Wordle,231,4,0.0
204,Michael,2022-02-04 08:41:31,Wordle 230 5/6\n\n🟨🟨⬛⬛⬛\n🟨🟩⬛⬛🟨\n⬛🟩🟩⬛🟨\n🟩🟩🟩🟩⬛\n...,Wordle,230,5,0.0
205,Danielle,2022-02-04 08:39:54,Wordle 230 4/6\n\n⬛🟨⬛⬛🟨\n⬛🟨🟨🟨⬛\n🟩🟨🟨🟨🟩\n🟩🟩🟩🟩🟩,Wordle,230,4,0.0


In [79]:
michaelwindf = michaeldf[michaeldf['score'].astype(str).str.isnumeric()]

In [80]:
michaelwindf

,player,game_num,score,LOSS


In [82]:
#michaeldf[michaeldf['score'].astype(str).str.isnumeric()]
michaelwindf = michaeldf[michaeldf['score'].astype(str).str.isnumeric()]

In [83]:
michaelwindf

,player,game_num,score,LOSS


In [96]:
###############
### imports ###
###############


import numpy as np
import pandas as pd
import requests
import os



import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, inspect


###########################
### pulling fresh texts ###
###########################


engine = db.create_engine('sqlite:////Users/michaelharnett/Library/Messages/chat.db')
connection = engine.connect()
metadata = db.MetaData()

orig_df = pd.read_sql("select distinct m.rowid ,m.is_from_me IsFromMe ,case when m.is_from_me = 1 then m.account else h.id end as FromPhoneNumber ,datetime((m.date / 1000000000) + 978307200, 'unixepoch', 'localtime') as TextDate, m.text MessageText,c.display_name RoomName from message as m left join handle as h on m.handle_id = h.rowid left join chat as c on m.cache_roomnames = c.room_name left join chat_handle_join as ch on c.rowid = ch.chat_id left join handle as h2 on ch.handle_id = h2.rowid where RoomName LIKE 'Wordle Fam%' order by m.date desc;", engine)






#############################
### making correct tables ###
#############################


df = orig_df[['FromPhoneNumber','TextDate','MessageText']] # taking only the fields I need
df = df.rename(columns = {'FromPhoneNumber':'player', 'TextDate':'date', 'MessageText':'message'}) #fixing names


# Renaming changing the Id from phone number to our names
df.loc[df.player == '+12014523662', 'player'] = 'Danielle'
df.loc[df.player == '+12017253072', 'player'] = 'Dad'
df.loc[df.player == '+15514868670', 'player'] = 'Chris'
df.loc[df.player == 'E:michaelcharnett@gmail.com', 'player'] = 'Michael'


# Creating columns
df['wordle_check'] = ''
df['game_num'] = ''
df['score'] = ''

# fixing indexes
df = df.dropna()
df = df.reset_index()
df =df.drop(columns = ['index'])

#updating values
for i in range(1,len(df)):
    df.loc[i,'wordle_check'] = df.loc[i,'message'][:6]
    df.loc[i,'game_num'] = df.loc[i,'message'][7:10]
    df.loc[i,'score'] = df.loc[i,'message'][11:14]
    

    
# creating wordledf, df of just wordle messages
wordledf = df    
wordledf = df[df.wordle_check == 'Wordle']
wordledf = wordledf.drop(columns = ['message','date','wordle_check'])



# changing scores to remove the denominator, a they're all out of 6
wordledf = wordledf.reset_index()
wordledf = wordledf.drop(columns = ['index'])   ##for some reason, it works if I keep resetting index

for i in range(len(wordledf)):
    wordledf.loc[i, 'score'] = wordledf.loc[i, 'score'][0]




#######################
### cleaneingtables ###
#######################


## for some reason was erroring..not needed these columns were dropped elsewhere
# # removing unneeded columns
# wordledf = wordledf.drop(columns = ['message', 'date'])
# wordledf = wordledf.drop(columns = ['wordle_check'])

# changing data types
wordledf.player = wordledf.player.astype('str')
wordledf.game_num = wordledf.game_num.astype('int')
wordledf.score = wordledf.score.astype('str')




# setting wins/loss in new column    -----------------MAY BE UNNECCESRY 
wordledf[wordledf.score == 'x']
wordledf[wordledf.score == 'X']    #there seems to be case inconsistencies 

wordledf.loc[wordledf.score == 'x','LOSS'] = 1
wordledf.loc[wordledf.score != 'x','LOSS'] = 0

wordledf.loc[wordledf.score == 'X','LOSS'] = 1
wordledf.loc[wordledf.score != 'X','LOSS'] = 0




# individualized tables
michaeldf = wordledf[wordledf.player == 'Michael']
danielledf = wordledf[wordledf.player == 'Danielle']
chrisdf = wordledf[wordledf.player == 'Chris']
daddf = wordledf[wordledf.player == 'Dad']


# ordering the tables
michaeldf = michaeldf.sort_values(by='game_num', ascending = False)
danielledf = danielledf.sort_values(by='game_num', ascending = False)
chrisdf = chrisdf.sort_values(by='game_num', ascending = False)
daddf = daddf.sort_values(by='game_num', ascending = False)

# making a df of just wins to compare against
michaelwindf = michaeldf[michaeldf['score'].astype(str).str.isnumeric()]
dadwindf = daddf[daddf['score'].astype(str).str.isnumeric()]
daniellewindf = danielledf[danielledf['score'].astype(str).str.isnumeric()]
chriswindf = chrisdf[chrisdf['score'].astype(str).str.isnumeric()]



#######################################################################################
########    REALIZING, tables should be made and exprted. thsi can be dane after ######
########################################################################################
#### Win percentages 
#### michael_winpercent = (len(michaelwindf)/len(michaeldf)*100)
#### dad_winpercent = (len(dadwindf)/len(daddf)*100)
#### danielle_winpercent = (len(daniellewindf)/len(danielledf)*100)
#### chris_winpercent = (len(chriswindf)/len(chrisdf)*100)


##############################
#### saving dfs to folder ####
##############################

# michael
michaeldf.to_csv('tables/michaeldf.csv', index = False)
michaelwindf.to_csv('tables/michaelwindf.csv', index = False)

# danielle
danielledf.to_csv('tables/danielledf.csv', index = False)
daniellewindf.to_csv('tables/daniellewindf.csv', index = False)

#chris
chrisdf.to_csv('tables/chrisdf.csv', index = False)
chriswindf.to_csv('tables/chriswindf.csv', index = False)

#dad
daddf.to_csv('tables/daddf.csv', index = False)
dadwindf.to_csv('tables/dadwindf.csv', index = False)

In [97]:
michaelwindf

,player,game_num,score,LOSS
2,Michael,286,6,0.0
7,Michael,285,4,0.0
10,Michael,284,2,0.0
13,Michael,283,5,0.0
17,Michael,282,4,0.0
24,Michael,280,5,0.0
28,Michael,279,4,0.0
33,Michael,278,5,0.0
41,Michael,276,3,0.0
53,Michael,271,6,0.0


In [98]:
michael_winpercent = (len(michaelwindf)/len(michaeldf)*100)
dad_winpercent = (len(dadwindf)/len(daddf)*100)
danielle_winpercent = (len(daniellewindf)/len(danielledf)*100)
chris_winpercent = (len(chriswindf)/len(chrisdf)*100)

In [99]:
michael_winpercent

92.0

In [100]:
michaeltable = pd.read_csv('tables/michaeldf.csv')

In [101]:
michaeltable

,player,game_num,score,LOSS
0,Michael,286,6,0.0
1,Michael,285,4,0.0
2,Michael,284,2,0.0
3,Michael,283,5,0.0
4,Michael,282,4,0.0
5,Michael,280,5,0.0
6,Michael,279,4,0.0
7,Michael,278,5,0.0
8,Michael,277,X,1.0
9,Michael,276,3,0.0
